In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 데이터 읽기
normal_df = pd.read_csv("normal.csv")
abnormal_df = pd.read_csv("abnormal.csv")

# 데이터 병합
df = pd.concat([normal_df, abnormal_df], ignore_index=True)

# 특성과 레이블 분리
X = df.drop(["class", "motor_id"], axis=1)
y = df["class"]

# 데이터 정규화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 학습과 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=1
)

# SVM 모델 정의
svm_model = SVC(probability=True)

# Hyper-parameter 그리드 서치 설정 (범위 축소)
param_grid = {
    "C": [0.1, 1, 10],
    "gamma": [0.1, 1, 10],
}

# GridSearchCV를 사용하여 최적의 hyper-parameters 찾기
svm_grid_search = GridSearchCV(svm_model, param_grid, scoring="roc_auc", cv=5)
svm_grid_search.fit(X_train, y_train)

# 최적의 hyper-parameters를 사용하여 SVM 모델 학습
best_svm_model = svm_grid_search.best_estimator_
svm_y_pred = best_svm_model.predict(X_test)

# 성능 측정
accuracy = accuracy_score(y_test, svm_y_pred)
precision = precision_score(y_test, svm_y_pred)
recall = recall_score(y_test, svm_y_pred)
f1 = f1_score(y_test, svm_y_pred)
roc_auc = roc_auc_score(y_test, svm_y_pred)

# 결과 DataFrame 생성
svm_results_df = pd.DataFrame({
    'Model': ['SVM'],
    'Accuracy': [accuracy],
    'Precision': [precision],
    'Recall': [recall],
    'F1': [f1],
    'ROC-AUC': [roc_auc]
})

# 결과 출력
print(svm_results_df)


  Model  Accuracy  Precision    Recall        F1   ROC-AUC
0   SVM  0.937221    0.94718  0.925455  0.936192  0.937166


In [2]:
# 최적의 hyper-parameters 출력
best_params = svm_grid_search.best_params_
print("Best Hyperparameters for SVM:", best_params)

Best Hyperparameters for SVM: {'C': 10, 'gamma': 0.1}
